<a href="https://colab.research.google.com/github/mdallas1/shared_code/blob/main/L5_1_lu_factorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt install octave

# Chapter 5: Linear Systems

One of the most fundamental problems in numerical analysis is solving a linear system. Linear systems show up in many branches of science, and many techniques for nonlinear equations involve the solution of a linear system of equations as a subroutine. A general linear system of equations with $n$ unknowns $x_1,...,x_n$ and $m$ equations takes the form
$$
\begin{align}
c_{11}x_1+c_{12}x_2+\cdots+c_{1n}x_n &= b_1 \\
c_{21}x_1+c_{22}x_2+\cdots+c_{2n}x_n &= b_2 \\
&\vdots \\
c_{m1}x_1+c_{m2}x_2+\cdots+c_{mn}x_n &= b_m \\
\end{align}
$$

Here the numbers $c_{ij}$ with $1\leq i\leq m$ and $1\leq j\leq n$ are known constants, as are the numbers $b_1$,...,$b_m$. It is convenient to express linear systems in matrix form. If we define the $m\times n$ matrix
$$
\begin{align}
A = \begin{bmatrix}
c_{11} &c_{12} &\dots &c_{1n}\\
c_{21} &c_{22} &\dots & c_{2n} \\
\vdots &\vdots &\dots &\vdots \\
c_{m1} &c_{m2} &\dots &c_{mn}
\end{bmatrix}
\end{align}
$$
then by the definition of matrix-vector multiplication, we can express the system of equations above as
$$
\begin{align}
A\mathbf{x} = \mathbf{b}, \hspace{1em} \text{ where } \hspace{1em} \mathbf{x} = \begin{bmatrix} x_1 \\ x_2 \\ \vdots \\x_n\end{bmatrix}, \hspace{1em}\mathbf{b} = \begin{bmatrix}  b_1 \\ b_2 \\ \vdots \\ b_m \end{bmatrix}
\end{align}
$$

The fundamental problem tackled in this chapter is the solution of $A\mathbf{x}=\mathbf{b}$ when $A$ is a square matrix, i.e., it has the same number of rows and columns so that $m=n$. We will also discuss how we can handle *overdetermined systems* for which $m > n$.
## Representative Problems

Our book develops several problems in detail. Here, we will only highlight those that we will revisit as examples later.

1. **(Hydraulic network)** In this example, we are given a hydraulic network and we want to determine the pressure at the internal nodes. Assuming our system has no leaks, conservation of mass tells us that the net flow rate at each node must be zero. This leads to the linear system
seen in ex5_5.m below.

2. **(Spectrometry)** Consider a gas mixture with $n$ unknown components. Using a mass spectrometer, one obtains a series of peaks that can be used to analyze the mixture. One model of these peaks says that the $i$th peak of height $h_i$ is a linear combination of the partial pressures $p_j$, $j=1,...,n$. That is, the pressure that component $j$ would exhibit were it the sole component of the gas. This model can be expressed as
$$
\begin{align}
\sum_{j=1}^n s_{ij}p_j = h_i, \hspace{1em} i=1,...,n.
\end{align}
$$
This is a linear system with $n$ equations and $n$ unknowns, and can be expressed as a matrix equation (see Example 5.3 below).

3. **(Capillary Network)** The model from problem 1 can be applied to model capillary networks in the circulatory system. The main difference between this problem 1 is that here the capillary network is much less interconnected. This leads to what is known as a **sparse** matrix (and also symmetric). We will see one way to handle this problem in Example 5.7.

# How not to solve a large linear system

From our linear algebra review, we've seen that a linear system $A\mathbf{x}=\mathbf{b}$ has a solution if and only if $A$ is nonsingular. In this case $\det(A)\neq 0$, which means we could use [Cramer's rule](https://en.wikipedia.org/wiki/Cramer%27s_rule#Finding_inverse_matrix) to find that
$$
\begin{align}
x_i = \dfrac{\det(A_i)}{\det(A)}
\end{align}
$$
for $i=1,...,n$. Here, $A_i$ is the matrix obtained by replacing the $i$th column of $A$ with $b$. Were we to compute these determinants using cofactor expansion, since we have a total of $n+1$ determinants to compute, the cost of solving for $\mathbf{x}$ using Cramer's rule would grow like $\mathcal{O}((n+1)!)$. So for large $n$, the cost is about $3(n+1)!$. A typical personal computer these days can compute about $10^9$ floating point operations per second. On such a machine, it would take about $10^{-1}$ seconds to solve a $10\times 10$ matrix. To solve a $20\times 20$ system, however, would take about 4860 years.

We, however, live in the era of exascale ($10^{18}$ flops) computing. As of 2024, the fastest computer in the world is El Capitan at Lawrence Livermore. They can manage an astounding $1.7\times 10^{18}$ floating point operations a second. Giving this machine a system with 20 unknowns is a joke. It can solve it using Cramer's rule in about 2.5 minutes, but if the system has just 30 uknowns, then El Capitan would take $7.8\times 10^8$ years to solve it using Cramer's rule. That's just under a billion years: a comically long time. This tells us that Cramer's rule is utterly impractical to solve even modestly sized systems.

This motivates the development of more efficient techniques, and these can be categorized broadly as *direct* and *iterative* methods. Direct methods are those that will solve the system in a finite number of steps, whereas iterative methods would theoretically take infinitely many steps to converge. The latter will feel similar to Newton's method and other methods we came across while studying nonlinear equations. We first turn our attention to a direct method known as LU factorization, which for decades has been a popular, reliable linear solver. First, we will review Gauss elimination.


# Gauss Elimination

We will review the key parts of Gauss elimination by considering the $3\times 3$ system
$$
\begin{align}
2x_1 + 2x_2 + 2x_3 &= 2\\
x_1 + 4x_2 + 1x_3 &= 5\\
x_1+2x_2+3x_3 &= 10.
\end{align}
$$
Our goal is to determine the three unknowns $x_1$, $x_2$, and $x_3$. You should convince yourself that if we

1. swap two equations in the system,
2. multiply any one equation by a nonzero number, or
3. add a multiple of one equation to another,

then we do *not* change the set of solutions to the system. We can make our lives a bit easier by expressing the system above in matrix notation. If we view the left hand side as a matrix-vector product, with the vector being $\mathbf{x} = [x_1 , x_2 , x_3]^T$, then the system is equivalent to

$$
\begin{align}
\begin{bmatrix} 2 &2 &2 \\ 1 &4 &1 \\ 1 &2 &3 \end{bmatrix}
\begin{bmatrix} x_1 \\ x_2 \\x_3 \end{bmatrix}
=
\begin{bmatrix} 2 \\5 \\10 \end{bmatrix}
\end{align}
$$

If you have studyed linear algebra before, you may expect us to form the augmented matrix next and then start eliminating. This is not what we will do here. Instead, we will apply Gauss elimination to turn all the entries below the diagonal to zeros.
In other words, our goal is to use Gauss eliminiation to turn $A$ into an upper triangular matrix.

Since the rows of $A$ correspond to the equations in our linear system, the three "legal" operations we could perform on the system of equations, swapping equations, multiplying an equation by a nonzero constant, and adding a multiple of one equation to another, carry over to operations we can perform on the *rows* of $A$.

We can use the third operation, adding a multiple of one row to another, to eliminate nonzero entries below the main diagonal. Let's start with the first column. Adding $(-1/2)[2, 2, 2]$ to  $[1, 4 ,1]$ and $[1, 2, 3]$  yields

$$
\begin{align}
\begin{bmatrix} 2 &2 &2 \\ 0 &3 &0 \\ 0 &1 &2 \end{bmatrix}
\end{align}
$$

Now, adding $(-1/3)[0, 3, 0]$ to the last row $[0, 1, 2]$ yields

$$
\begin{align}
\begin{bmatrix} 2 &2 &2 \\ 0 &3 &0 \\ 0 &0 &2 \end{bmatrix}
\end{align}
$$

It might not be clear why this was useful. The situation will be much more clear if we express the elimination steps with matrix multiplication.

## Elementary Row operations

Each row operation applied to $A$ can be expressed as $EA$, where $E$ is an *elementary matrix*. These are matrices generated by applying precisely one of the three row operations to the identity matrix. The first row operation we applied above was to add $(-1/2) $row 1 to row 2. If we apply this operation to the $3\times 3$ identity matrix, we get
$$
\begin{align}
E_1 = \begin{bmatrix} 1 &0 &0 \\ -1/2 &1 &0 \\ 0 &0 &1 \end{bmatrix}
\end{align}
$$
We then added $(-1/2)$row 1 to row 3. Applying this operation to the $3\times 3$ identity gives
$$
\begin{align}
E_2 = \begin{bmatrix} 1 &0 &0 \\ 0 &1 &0 \\ -1/2 &0 &1 \end{bmatrix}
\end{align}
$$
Lastly, we added $(-1/3)$row 2 to row 3. Which corresponds to the elementary matrix
$$ E_3 = \begin{bmatrix} 1 &0 &0\\ 0 &1 &0\\ 0 &-1/3 &1\end{bmatrix} $$

By the definition of matrix multiplication, multiplying $A$ on the left by $E_1$ performs whatever operation we applied to $I$ to get $E_1$ to $A$. That is, $E_1A$ is equivalent to multiplying row 1 of $A$ by $-1/2$ and adding it to row 2. Likewise for $E_2$ and $E3$. Thus, the operations we applied before by be written as
$$
\begin{align}
E_3E_2E_1A = \begin{bmatrix} 2 &2 &2 \\ 0 &3 &0 \\ 0 &0 &2 \end{bmatrix}
\end{align}
$$

Observe that the matrices $E_1$, $E_2$, and $E_3$ are lower triangular, and the product of lower triangular matrices yields another lower triangular matrix. In our case,
$$
\begin{align}
\hat{L} = E_3E_2E_1 = \begin{bmatrix} 1 &0 &0 \\ -1/2 &1 &0 \\ -1/3 &-1/3 &1 \end{bmatrix}
\end{align}
$$

Here's a question: does $\hat{L}$ have an inverse? If yes, what is it?

You can verify that

$$ L = E_1^{-1}E_2^{-1}E_3^{-1} = \begin{bmatrix} 1 &0 &0 \\ 1/2 &1 &0 \\ 1/2 &1/3 &1 \end{bmatrix}$$

is the inverse of $\hat{L}$. Thus, if we write
$$ U = \begin{align}
\begin{bmatrix} 2 &2 &2 \\ 0 &3 &0 \\ 0 &0 &2 \end{bmatrix}
\end{align}$$ then

$$ \hat{L}A = U \implies A = LU.$$

# LU Factorization

What we've just shown through example is that performing Gaussian elimination with matrix multiplication allows us to write $A = LU$, where $L$ is a lower triangular matrix, and $U$ is an upper triangular matrix. When we write a matrix $A$ as a product of two or more matrices, we call this a *factorization* or a *decomposition* of $A$. In this case, since $A$ is written as the product of a lower triangular and upper triangular matrix, we call this an $LU$ factorization, or decomposition, of $A$. How does this help us solve a linear system $A\mathbf{x}=\mathbf{b}$?

Let's assume that $A$ is $n\times n$, nonsingular, and that we can write $A = LU$. In other words, assume that $A$ has an LU factorization. Then
$$ A\mathbf{x} = \mathbf{b} \iff LU\mathbf{x} = \mathbf{b}.$$

If we write $U\mathbf{x} = \mathbf{y}$, then $\mathbf{x}$ solves $A\mathbf{x}=\mathbf{b}$ if and only if $\mathbf{x}$ and $\mathbf{y}$ solve

$$
\begin{align}
L\mathbf{y} &= \mathbf{b} \\
U\mathbf{x} &= \mathbf{y}
\end{align}
$$

We have thus reduced the problem of solving $A\mathbf{x}=\mathbf{b}$ to solving two *triangular* systems. These are easy to solve, since we can just attack one equation at a time. Denote the entries of $L$ as $l_{ij}$ for $1\leq i,j\leq n$. The first row of $L\mathbf{y} =\mathbf{b}$ corresponds to the equation $$l_{11}y_1 = b_1$$. Thus $y_1 = b_1/l_{11}$. The second row is $$l_{21}y_1 + l_{22}y_2 = b_2.$$ We just computed $y_1$, so we can simply solve for $y_2$ to get $$y_2 = \dfrac{b_2 -l_{21}y_1}{l_{22}}.$$
Continuing this way allows to solve for $\mathbf{y}$. This strategy gives us the *forward substitution algorithm*
> **Forward Substitution Algorithm for $Lx=b$**
$$
\begin{align}
y_1 &= \dfrac{b_1}{l_{11}} \\
y_i &= \dfrac{1}{l_{ii}}\left(b_i - \sum_{j=1}^{i-1} l_{ij}y_j\right), i = 2,3,...,n.
\end{align}
$$

Once we've applied forward subsitution to solve $L\mathbf{y}=\mathbf{b}$ for $\mathbf{y}$, we can solve $U\mathbf{x}=\mathbf{y}$. This is an upper-triangular system. We'll let $u_{ij}$ denote the $ij$th entry of $U$. The $n$th equation of $U\mathbf{x}=\mathbf{y}$ is
$$x_nu_{nn} = y_n$$. We can easily solve this to get $x_{nn} = y_n / u _{nn}$. The $n-1$st equation is
$$ x_{n-1}u_{n,n-1} + x_nu_{nn} = y_{n-1}$$
which, since we now know $x_n$, can easily be solved for $x_{n-1}$:
$$ x_{n-1} = \dfrac{1}{u_{n,n-1}}\left( y_{n-1} - x_{n-1}u_{n,n-1}\right)$$

Continuing up, we obtain the *backwards substitution algorithm*:
> **Backward Substitution Algorithm for $Ux=y$**
$$
\begin{align}
x_n &= \dfrac{y_n}{u_{nn}} \\
x_i &= \dfrac{1}{u_{ii}}\left(y_i - \sum_{j=i+1}^{n} u_{ij}x_j\right), i= n-1,...,1.
\end{align}
$$

It's not too difficult to show that the cost of solving a triangular system using the backwards or forwards subsitution algorithm, as appropriate, is $\mathcal{O}(n^2)$. So if we can find an efficient way to compute $L$ and $U$ given $A$, we will have a much faster way to solve $A\mathbf{x}=\mathbf{b}$ compared to Cramer's rule.

Below are two programs implementing forwards and backwards subsitution.


In [ ]:
#@title fwd_sub.m
%%writefile fwd_sub.m

% solve random problem
n=10;
A = 5*rand(n); L = tril(A);
b = rand(n,1);

% or define particular problem
% L = [];
% b = [];

[m,n] = size(L);  x = zeros(n,1);

if abs(L(n,n)) < 1e-12
  error("Diagonal zero error")
end

x(1) = b(1)/L(1,1);

for i = 2:n
  if abs(L(i,i)) < 1e-12
      error("Diagonal zero error")
  end
  x(i) = ( b(i) - L(i,1:i-1)*x(1:i-1) )/L(i,i);
end

fprintf("Error: %.2e\n",norm(x - (L\b))) % check error

In [ ]:
!octave -W fwd_sub.m

In [ ]:
#@title bck_sub.m
%%writefile bck_sub.m

% solve random problem
n=10;
A = 5*rand(n); U = triu(A);
b = rand(n,1);

% or define particular problem
 U = [];
 b = [];

[m,n] = size(U);  x = zeros(n,1);

if abs(U(n,n)) < 1e-12
  error("Diagonal zero error")
end

x(n) = b(n)/U(n,n);

for i = n-1:-1:1
  if abs(U(i,i)) < 1e-12
      error("Diagonal zero error")
  end
  x(i) = ( b(i) - U(i,i+1:n)*x(i+1:n) )/U(i,i);
end

fprintf("Error: %.2e\n",norm(x - (U\b))) % check error

In [ ]:
!octave -W bck_sub.m

# Computing the LU Factorization

In the example above, we computed $L$ and $U$ using Gauss elimination. By generalizing the same procedure we used above, we obtain the LU factorization algorithm. The code below this cell helps clarify what's going on behind the scenes, but Octave has a built in function lu.m that efficiently computes the LU factorization of a matrix A. We will discuss this algorithm once we discuss pivoting. For now, you can use the authors' program lugauss.m in shared_code/textbook_algorithms or the code in the cells below.

Before looking at the code, it will be helpful to do the calculations by hand for a small matrix. Let
$$ A  = \begin{bmatrix} a_{11} &a_{12} &a_{13} \\ a_{21} &a_{22} &a_{23} \\
a_{31} &a_{32} &a_{33}\end{bmatrix}$$.
Compute the following.

1. $$\begin{bmatrix} 1 &0 &0 \\ -\frac{a_{21}}{a_{11}} &1 &0 \\ -\frac{a_{31}}{a_{11}} &0 &1\end{bmatrix}\begin{bmatrix} a_{11} &a_{12} &a_{13} \\ a_{21} &a_{22} &a_{23} \\
a_{31} &a_{32} &a_{33}\end{bmatrix}$$  
If the $ij$th entry of the product is nonzero, but different from $a_{ij}$, you may just write $a_{ij}'$.

2. Letting $L_1$ denote the lower triangular matrix form step 1, you should obtain
$$L_1A  = \begin{bmatrix} a_{11} &a_{12} &a_{13} \\ 0 &a'_{22} &a'_{23} \\
0 &a'_{32} &a'_{33}\end{bmatrix}$$,
where the $'$ indicates that the corresponding entry is different from that of $A$. Now compute
$$ L_2L_1A = \begin{bmatrix} 1 &0 &0 \\ 0 &1 &0 \\ 0 &-\frac{a_{32}'}{a_{22}} &1\end{bmatrix}\begin{bmatrix} a_{11} &a_{12} &a_{13} \\ 0 &a'_{22} &a'_{23} \\
0 &a'_{32} &a'_{33}\end{bmatrix}$$  
Similar to step 1, if the entries in $L_1A$ change, but are nonzero, you can indicate this with $''$.

You should find that
$$ L_2L_1A = \begin{bmatrix} a_{11} &a_{12} &a_{13} \\ 0 &a'_{22} &a'_{23} \\
0 &0 &a''_{33} \end{bmatrix} := U $$

3. Explain in one sentence why $L_2$ and $L_1$, respectively, have an inverse, and identify that inverse. **Hint:** What row operations do $L_1$ and $L_2$ perform? Can you undo these operations?

4. Let $L = L_1^{-1}L_2^{-1}$, and conclude that
$$ A = LU$$, where $L$ is lower triangular and $U$ is upper triangular. What are $L_2^{-1}$ and $L_1^{-1}$? What's $L$ explicitly?

Notice that when multiplying by $L_1$ only affected the the entries below $a_{11}$, and the entries of the submatrix obtained by removing the first row and first column from $A$. Similarly, $L_2$ only affects the entries below $a_{22}'$, and the entries in the submatrix formed by removing the first and second row,and first and second column.



In [ ]:
#@title lu_fact.m
%%writefile lu_fact.m

A = 5*rand(5) % define random 5x5 matrix here

% Computes, if possible, the LU factorization of A
[m,n] = size(A);

if m ~= n
  error("Matrix must be square.")
end

L = eye(n); % identity matrix of size n

for k = 1:n-1

  if abs(A(k,k)) < 1e-10
    error("Zero diagonal error.")
  end

  for i = k+1:n
    L(i,k) = A(i,k)/A(k,k); % define L(i,k) to zero all entries below A(k,k) using Gauss elimination

    for j = k+1:n
      A(i,j) = A(i,j) - L(i,k)*A(k,j); % this is the product of L(:,k) with the submatrix A(k+1:n,k+1:n) and corresponds to the a_{ij}' terms in
      % the example above.
    end

  end

end

U = triu(A); % extract uppper triangular part of A

L,U, L*U

In [ ]:
!octave -W lu_fact.m

In [ ]:
#@title efficient_lu_fact.m
%%writefile efficient_lu_fact.m

% Computes LU factorization of A using only one matrix, which is more memory efficient.
n=5;
A = 2*rand(n)

% Computes, if possible, the LU factorization of A
for k = 1:n-1

  if abs(A(k,k)) < 1e-10
    error("Zero diagonal error.")
  end

  for i = k+1:n
    A(i,k) = A(i,k)/A(k,k); % define L(i,k) to zero all entries below A(k,k) using Gauss elimination

    for j = k+1:n
      A(i,j) = A(i,j) - A(i,k)*A(k,j); % this is the product of L(:,k) with the submatrix A(k+1:n,k+1:n);
    end

    % b(i) = b(i) - A(i,k)*b(k) % uncomment if solving the system Ax=b with Gauss Elimination Method.

  end

end

tril(A,-1); triu(A);

% Test by extracting upper triangular part and lower triangular. Manually add 1's to diagonal of L.
L = tril(A,-1); for i = 1:n; L(i,i) = 1; end;
U = triu(A);
L,U,L*U


In [ ]:
!octave -W efficient_lu_fact.m

# Gauss Elimination Method

Although we did not work with an augmented matrix earlier when deriving the LU factorization, we certainly could have. All we have to do is add a line in the above program to apply the row operations to $\mathbf{b}$. Once we've gotten $A$ down to a upper triangular matrix, we can solve the resulting triangular system using backward subsitution. See the program on page 147 of our book for more details.

# Example 5.3 (Spectrometry)

Consider a gas mixture that yields the following seven strongest peaks after a spectroscopic inspection: $h_1 = 17.1$, $h_2 = 65.1$, $h_3 = 186.0$, $h_4 = 82.7$, $h_5 = 84.2$, $h_6 = 63.7$, and $h_7 = 119.7$. Suppose the total measured pressure is 38.78 $\mu$m of Hg. We want to compare this to the predicted pressure using the model
$$ \sum_{j=1}^n s_{ij}p_j = h_i$$
with $n=7$. The sensitivty coefficients, the $s_{ij}$'s, are given in the table below.  

|Peak | Hydrogen| Methane | Etilene | Ethane | Propylene | Propane | n-Pentane |
|---|---|---|---|---|---|---|---|
|Index | 1| 2 | 3 | 4 | 5 | 6 | 7 |
|1|16.87|0.1650|0.2019|0.3170|0.2340|0.1820|0.1100|
|2|0.0|27.70|0.8620|0.0620|0.0730|0.1310|0.1200|
|3|0.0|0.0|22.35|13.05|4.420|6.001|3.043|
|4|0.0|0.0|0.0|11.28|0.0|1.110|0.3710|
|5|0.0|0.0|0.0|0.0|9.850|1.1684|2.108|
|6|0.0|0.0|0.0|0.0|0.2990|15.98|2.107|
|7|0.0|0.0|0.0|0.0|0.0|0.0|4.670|

So far we've studied forwards subsitution, backwards substitution, and LU factorization. Determine which method is most appropriate here, and implement it below to solve for the unknown pressures $p_i$, $i=1,...,7$. You may use code from above by copying and pasting it into the cell below.

In [ ]:
%%writefile ex5_3.m

% Write your solution to example 5.3 here. You may follow the steps, or ignore them.

% Step 1: define the 7x7 matrix S using the data from the table above, and set b to be the vector
% whose entries are h_1, h_2, ..., h_7.

% Step 2: You should find the LU factorization to be the best choice. Compute the LU decomposition of S.

% Step 3: Solve the system Ly = b using forward subsitution.

% Step 4: Solve Ux = y using backward subsitution. What's your final value of x?

In [ ]:
# when finished writing your program ex5_3.m, run this cell to see what you get.
!octave -W ex5_3.m

# Computational Cost of LU factorization

Recall that if we can write $A=LU$, then we can reduce thet system $A\mathbf{x} =\mathbf{b}$ to the solution of two triangular systems, the cost of which grows like $\mathcal{O}(n^2)$. Now that we have an algorithm for computing the LU factorization, we can determine its cost. It can be shown that the algorithm above for computing the LU factorization requires about $2n^3/3$ floating point operations for large $n$.

Therefore, the cost of solving $A\mathbf{x}=\mathbf{b}$ by first computing the LU factorization of $A$ and then solving two triangular systems will grow like $\mathcal{O}(n^3)$. How does this compare to Cramer's rule? We saw earlier that, with Cramer's rule, even El Capitan would take just under a billion years to solve a system with 30 unknowns. If we instead had El Capitan solve such a system with the LU method, it would take about $10^{-14}$ seconds, essentially instantaneous. A system with a million unknowns could be solved in about a minute.

A more down-to-earth machine that can handle about $10^9$ flops can solve a system with 1000 unknowns using the LU method in about a minute. This is a huge improvement over Cramer's rule.